In [ ]:
import numpy as np

import pandas as pd
from pandas import read_csv
from pandas import datetime
from pandas import DataFrame
from pandas.plotting import autocorrelation_plot

import matplotlib.pyplot as plt

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.tsa.stattools as st


import seaborn as sns

from sklearn.metrics import mean_squared_error

In [ ]:
def tsplot(y, lags=None, title='', figsize=(14, 8)):
    '''Examine the patterns of ACF and PACF, along with the time series plot and histogram.
    
    Original source: https://tomaugspurger.github.io/modern-7-timeseries.html
    '''
    fig = plt.figure(figsize=figsize)
    layout = (2, 2)
    ts_ax   = plt.subplot2grid(layout, (0, 0))
    hist_ax = plt.subplot2grid(layout, (0, 1))
    acf_ax  = plt.subplot2grid(layout, (1, 0))
    pacf_ax = plt.subplot2grid(layout, (1, 1))
    
    y.plot(ax=ts_ax)
    ts_ax.set_title(title)
    y.plot(ax=hist_ax, kind='hist', bins=25)
    hist_ax.set_title('Histogram')
    smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
    smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
    [ax.set_xlim(0) for ax in [acf_ax, pacf_ax]]
    sns.despine()
    plt.tight_layout()
    return ts_ax, acf_ax, pacf_ax

In [ ]:
def ARIMA_plot_ki(series, 
               order,
               train_size_percent,
               scale=1):
    
    # Credit to Fong King Ian for providing this code

    # convert Series to DataFrame
    # remove duplicated rows except for most recent, convert to daily freq, fil blanks with prev observation
    X = series.rename("actual").to_frame() 
    X = X.loc[~X.index.duplicated(keep='last')].asfreq('d', 'ffill')
    
    # determine where the training set ends and the test set starts
    size = int(len(X) * train_size_percent)
    first_test_index = X.index[size]
        
    # forecast out-of-sample value using ARIMA
    for t in X[X.index >= first_test_index].index.tolist():
        # fit model with 'actual' values up to and excluding time t
        model = ARIMA(X[X.index < t]['actual'].values, order)
        model_fit = model.fit(disp=0)
        
        # forecast returns 3 arrays: 
        # out-of-sample forecast value, std error of the forecast and 
        # 2d array of the confidence interval for the forecast 
        
        output = model_fit.forecast()
        X.loc[t, 'predictions'] = output[0]  # output[0] contains forecast value
#        print('predicted = %f, expected = %f' % (output[0], X.at[t, 'actual']))
    
    # aligning predictions with correct time periods, removing rows without predictions
    X['predictions'] = X['predictions'].shift(-1)
    X.dropna(inplace = True)
    
    # MSE
    error = mean_squared_error(X['actual'].values, X['predictions'].values)
    print('Test MSE: %.3f' % error)
    
    # Scale to avoid exceeding maximum margin of plots
    test_scaled = X['actual'].values / scale
    predictions_scaled = X['predictions'].values / scale

    # plot
    plt.plot(test_scaled, color='blue', label='true values')
    plt.plot(predictions_scaled, color='red', label=f'estimated ARIMA{order}')
    plt.legend(loc='upper left')
    plt.show()

In [ ]:
def ARIMA_plot(series, 
               order,
               train_size_percent,
               scale = 1):
    
    # Split data points into train and test sets
    X = series
    size = int(len(X) * train_size_percent)
    train, test = X[0:size], X[size:len(X)]
    history = list(train)
    predictions = []
    
    # Forecast out-of-sample value using ARIMA
    for t in range(len(test)):
        model = ARIMA(history, order)     
        model_fit = model.fit(disp=0)
        # forecast returns 3 arrays: 
        # out-of-sample forecast value, std error of the forecast and 
        # 2d array of the confidence interval for the forecast 
        output = model_fit.forecast()
        # yhat contains forecast value
        yhat = output[0]
        predictions.append(yhat)
        obs = test[t]
        history.append(obs)
        print('predicted=%f, expected=%f' % (yhat, obs))
    
    # MSE
    error = mean_squared_error(test, predictions)
    print('Test MSE: %.3f' % error)
    
    # Scale to avoid exceeding maximum margin of plots
    test_scaled = [x/scale for x in test]
    predictions_scaled = [x/scale for x in predictions]

    # plot
    plt.plot(test_scaled, color = 'blue', label = 'true values')
    plt.plot(predictions_scaled, color='red', label = 'estimated ARIMA{}'.format(order))
    plt.legend(loc='upper left')
    plt.show()

In [ ]:
def parser(x):
    return datetime.strptime(x, '%Y-%m-%d')

In [ ]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.Series(diff)

In [ ]:
def fit_ARIMA(series, order):
    model = ARIMA(series, order)
    model_fit = model.fit(disp=0)
    return model_fit.summary()

In [ ]:
def AIC_BIC_HIQC(series, max_ar = 4, max_ma= 4):
    train_results = sm.tsa.arma_order_select_ic(series, ic=['aic', 'bic', 'hqic'], trend='nc', max_ar=max_ar, max_ma=max_ma)

    print('AIC', train_results.aic_min_order)
    print('BIC', train_results.bic_min_order)
    print('HQIC', train_results.hqic_min_order)

In [ ]:
def adf(series):
    adf_result = st.adfuller(series, store = True)
    print("ADF Test Results: ")
    print("Test Statistic: %.4f" % adf_result[0])
    print("p-value: %.10f" % adf_result[1])
    print("Critical Values: ")
    for key, value in adf_result[2].items():
        print('\t%s: %0.3f' % (key, value))

In [ ]:
def kpss(series):
    kpss_result = st.kpss(series, store=True)
    print("KPSS Test Results: ")
    print("Test Statistic: %.4f" % kpss_result[0])
    print("p-value: %.10f" % kpss_result[1])
    print("Critical Values: ")
    for key, value in kpss_result[2].items():
        print('\t%s: %0.3f' % (key, value))

======================================================================================================================

## Load Data Sets and Differencing Technique

In [ ]:
# Download Bitcoin's and Amazon's historical price in yahoo Finance as csv file
BitCoin = read_csv('BTC-USD.csv', 
                               header = 0, 
                               parse_dates = [0],  
                               index_col = 0, 
                               squeeze = True, 
                               date_parser = parser)

Amazon = read_csv('amzn.csv',  
                  header = 0, 
                  parse_dates = [0],
                  index_col = 0,
                  squeeze = True,
                 date_parser = parser)

AAPL_df = read_csv('AAPL.csv',
                  header = 0, 
                  parse_dates = [0],  
                  index_col = 0, 
                  squeeze = True, 
                  date_parser = parser)

temp_df = read_csv('daily-minimum-temperatures.csv', header=0)

In [ ]:
BitCoin.head()

In [ ]:
BitCoin.info()

In [ ]:
BitCoin.describe()

In [ ]:
tsplot(BitCoin['High'], title = 'Bitcoin High Price')

* Not all time series are stationary. In fact, most financial time series lack of stationarity.

* One method to 'stationarize' non-stationary time series is through differencing

* Notation: $\nabla = 1 - L$ where $L$ is the lag operator

* Advantage of differencing: 

    (1) No parameter estimation is required  

    (2) Can be repeated such that first difference $\nabla$ eliminates a linear trend whereas second difference $\nabla^2$ eliminates a quadratic trend

* Order of differencing $d$: $\nabla^d = (1-L)^d$

In [ ]:
# Difference Bitcoin high price by one interval
Bitcoin_high_diff = difference(BitCoin['High'], 2)
tsplot(Bitcoin_high_diff, title = 'Difference BitCoin High Price')

* Is the differenced Bitcoin High price series stationary?

In [ ]:
adf(Bitcoin_high_diff)

* Test statistics is less than all critical values. So according to ADF test, we can reject null hypothesis and conclude that the difference series is indeed stationary.

In [ ]:
tsplot(BitCoin['Low'], title = 'Bitcoin Open Price')

In [ ]:
# Difference Bitcoin low price by one interval
Bitcoin_low_diff = difference(BitCoin['Low'], 1)
tsplot(Bitcoin_low_diff, title = 'Difference Bitcoin Low Price')

In [ ]:
adf(Bitcoin_low_diff)

* Again, the test statistics is less than all critical values. So we reject null hypothesis and conclude that the difference series is stationary.

In [ ]:
tsplot(temp_df['Temp'], title = 'Temperature')

In [ ]:
# difference temperature by one interval
temp_df_diff = difference(temp_df['Temp'])
tsplot(temp_df_diff)

In [ ]:
adf(temp_df_diff)

* Test statistics is less than all critical values. So the difference series is stationary.

======================================================================================================================

## ARIMA$(p,d,q)$ Model

* A time series $\{X_t\}$ follows an ARIMA$(p,d,q)$ process if the $d^{th}$ differences $(1-L)^d X_t$ series is an ARMA($p,q$) process. Using lag operator $L$, it can expressed as 

$$\begin{equation}
  \phi(L)(1-L)^d X_t = c + \theta(L) \epsilon_t
\end{equation}$$

where $\phi(L) = 1 - \phi_1 L - \phi_2 L^2 - ... - \phi_p L^p$ and $\theta_q = 1 + \theta_1 L + \theta_2 L^2 - ... + \theta_q L^q.$

* In Python, $d=0,1,2.$ Higher value of $d$ is not accepted.

### Box Jenkins Method (ARIMA Model Identification)

* To estimate the parameters of a specific ARIMA$(p, d, q)$ model:

* To check on the appropriateness of the fitted model and improve it if needed.
 



1. **Model identification and model selection:** Determine the *order of differencing*, $d$

2. **Parameter estimation:** Study the ACF and PACF of the appropriately differenced series: $(1-L)^d X_t$, as these autocorrelation functions will provide indication for the choice of the order of autoregressive and the moving average components.  It is very beneficial to study the *theoretical* ACF and PACF of the autoregressive, moving average, and the mixed autoregressive and moving average processes.

3. **Model checking**: Test whether the estimated model conforms to the specifications of a stationary univariate process. In particular, the residuals should be independent of each other and constant in mean and variance over time. If the estimation is inadequate, we have to return to step one and attempt to build a better model.


* Let us recall the following table.

|  Process      |          ACF         |          PACF        |
|---------------|:--------------------:|:--------------------:|
| **AR(p)**     |    tails off         | cutoff after lag $p$ |
| **MA(q)**     | cutoff after lag $q$ |    tails off         |
| **ARMA(p,q)** |    tails off         |    tails off         |


* In general, the ACF of an autoregressive process is similar to the PACF of a moving average process, and vice versa.

* Keep in mind that these are theoretical properties. In practice, the estimated sample ACF and PACF can come with large variances, deviating from the underlying theoretical behavior. As such, it is prudent to recognize that these are  but broad characteristics, and it is quite possible that several candidate models are narrowed down and will need to be investigaged further in the later stage of the modeling process.

|  Process      |          ACF         |          PACF        |
|---------------|:--------------------:|:--------------------:|
| **ARIMA(p,d,0)**     |    tails off         | cutoff after lag $p$ |
| **ARIMA(0,d,q)**     | cutoff after lag $q$ |   tails off         |
| **ARIMA(p,d,q)** |   tails off         |   tails off         |


In [ ]:
# from IPython.display import Image
# Image(filename = 'ARIMA FLOW CHART.jpg', width = 200, height = 200 )

==================================================================================================================

## ARIMA Fitting

* In this section, we will fit ARIMA to BitCoin, Amazon and Apple high prices.

### Bitcoin High Price

In [ ]:
print(fit_ARIMA(BitCoin['High'], order=(4,2,3)))

In [ ]:
ARIMA_plot_ki(series = BitCoin['High'], order = (4,2,3), train_size_percent = 0.9, scale = 1000)

In [ ]:
AIC_BIC_HIQC(BitCoin['High'])

=====================================================================================================================

### Amazon High Price

In [ ]:
print(fit_ARIMA(Amazon['High'], order = (1,1,1)))

In [ ]:
ARIMA_plot_ki(series = Amazon['High'], order = (2,1,1), train_size_percent = 0.9, scale = 1000)

In [ ]:
AIC_BIC_HIQC(Amazon['High'])

=====================================================================================================================

### Apple High Price

In [ ]:
tsplot(AAPL_df['High'], title = 'Apple High Price', lags = 30)

In [ ]:
print(fit_ARIMA(AAPL_df['High'], order = (5,2,0)))

In [ ]:
ARIMA_plot_ki(series = AAPL_df['High'], order = (5,2,0), train_size_percent = 0.8)

In [ ]:
AIC_BIC_HIQC(AAPL_df['High'])

========================================================================================================================

### Residual Diagnostic

* The 'residuals' in a time series are what is left over after fitting a model. More precisely, residual is the difference between true and predicted value. 

* For many (but not all) time series models, the **residuals** are equal to the difference between the observations and the corresponding fitted values: 

$$\epsilon_t = X_t - \hat{X_t}$$

* Residuals are useful in checking whether a model has adequately captured the information in the data. 

* A good forecasting model will yield residuals with the following properties: 


1. $\textbf{The residuals are uncorrelated}$. If there are correlations between residuals, then there is information left in the residuals which shoud be used in computing forecasts. This can be done through inspecting acf plot.

2. $\textbf{The residuals have zero mean}$. If the residuals have a mean other than zero, then the forecasts are biased. This can be done in various ways, say, histogram.

3. $\textbf{The residuals follow a normal distribution}.$


In [ ]:
# residual of Apple's stock prices

model = ARIMA(AAPL_df['High'], order = (5,2,0))
model_fit = model.fit(disp=0)
residual = model_fit.resid
tsplot(residual, title = 'Residuals', lags = 30)

* To check normality, one can use Jarque-Bera test.

In [ ]:
# determine whether residuals follow a normal distribution
from scipy import stats

jb = stats.jarque_bera(residual)    
print("Jarque-Bera Test Results: ")
print("Test Statistic: %.4f" % jb[0])
print("p-value: %.10f" % jb[1])

* For Jarque-Bera test, the null hypothesis is that skewness is zero and excess kurtosis is zero, that is, the data follows a normal distribution. Alternative hypothesis would be otherwise.

* Since $p$-value is less than $0.05,$ we should reject null hypothesis. In other words, residuals are not normally distributed.